### TL;DR

Hice esto como en 2 horas, **es un experimento** y nada oficial

In [105]:
import pandas as pd
import category_encoders as ce
from bayes_opt import BayesianOptimization
from keras.models import Sequential
from keras.layers import Dense, InputLayer
from sklearn.preprocessing import MinMaxScaler

In [106]:
def test_data():
    data = pd.read_csv("train.csv")
    data.drop(labels=['PassengerId','Name','Ticket','Cabin'], axis=1, inplace=True)
    encoder = ce.TargetEncoder(cols=['Embarked','Sex','Pclass'])
    data = encoder.fit_transform(data, data['Survived'])
    data = data.dropna()
    X = data[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
    scaler = MinMaxScaler()
    X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    y = data['Survived']
    return X,y

### AutoDrill

Por ahora solo funciona para modelos de clasificación

In [117]:
class AutoDrill(object):
    input_dim=None
    num_classes=None
    num_models=None
    x = None
    y = None
    num_iter = None
    init_points = None
    best = None
    
    def __init__(self, num_iter, init_points):
        self.num_iter = num_iter
        self.init_points = init_points
        
    def _set_params(self, num_hidden, num_neurons):
        params = {
            'num_hidden':int(num_hidden),
            'num_neurons':int(num_neurons)
        }
        return params
        
    def _build_model(self, **kwargs):
        num_hidden = kwargs['num_hidden']
        num_neurons = kwargs['num_neurons']
        model = Sequential()
        model.add(InputLayer(input_shape=(self.input_dim,)))
        for i in range(0, num_hidden):
            model.add((Dense(num_neurons,activation="relu")))
        model.add(Dense(self.num_classes, activation="softmax"))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
        return model
    
    def _run_model(self, num_hidden, num_neurons):
        params = self._set_params(num_hidden, num_neurons)
        m = self._build_model(**params)
        h = m.fit(self.x.values, self.y.values, epochs=40, batch_size=16, verbose=0, shuffle=False)
        v = h.history['categorical_accuracy'][-1]
        return v
    
    def fit(self, x=None, y=None):
        self.input_dim = x.shape[1]
        self.num_classes = y.nunique()
        y = pd.get_dummies(y)
        self.x = x
        self.y = y
        opt = BayesianOptimization(self._run_model,{'num_hidden':(1,10),
                                                    'num_neurons':(self.input_dim, 2*self.input_dim)})
        opt.maximize(init_points=self.init_points, n_iter=self.num_iter)
        self.best = opt.res['max']

### Preparamos los datos

In [118]:
X,y = test_data()

In [122]:
X.head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0.0,0.0,0.271174,0.2,0.0,0.014151,0.0
1,1.0,1.0,0.472229,0.2,0.0,0.139136,1.0
2,0.0,1.0,0.321438,0.0,0.0,0.015469,0.0
3,1.0,1.0,0.434531,0.2,0.0,0.103644,0.0
4,0.0,0.0,0.434531,0.0,0.0,0.015713,0.0


In [123]:
y.head(5)

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

## Uso de AutoDrill

definimos el numero de init_points y el numero de iteraciones que queremos hacer

In [119]:
drill = AutoDrill(init_points=10, num_iter=15)

In [121]:
drill.fit(X,y)

Initialization
------------------------------------------------------------
 Step |   Time |      Value |   num_hidden |   num_neurons | 
    1 | 00m07s |    0.81373 |       5.5256 |       10.5285 | 
    2 | 00m06s |    0.82493 |       5.1018 |       11.2169 | 
    3 | 00m07s |    0.83473 |       4.2610 |       13.0372 | 
    4 | 00m06s |    0.82633 |       3.6222 |       13.4036 | 
    5 | 00m05s |    0.82913 |       4.0077 |        9.8845 | 
    6 | 00m09s |    0.83473 |       9.5393 |       13.0393 | 
    7 | 00m06s |    0.82773 |       2.3861 |        9.7313 | 
    8 | 00m08s |    0.81793 |       6.5915 |        9.8558 | 
    9 | 00m07s |    0.82353 |       5.4578 |        9.9827 | 
   10 | 00m08s |    0.82073 |       8.9239 |        9.9201 | 


/anaconda2/envs/matias/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-6.1141185e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 62, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)


Bayesian Optimization
------------------------------------------------------------
 Step |   Time |      Value |   num_hidden |   num_neurons | 
   11 | 00m10s |    0.81373 |      10.0000 |        7.0000 | 
   12 | 00m11s |    0.81933 |       1.0000 |        7.0000 | 
   13 | 00m12s |    0.81092 |       5.6177 |        7.0000 | 
   14 | 00m10s |    0.82073 |       1.0000 |       12.4417 | 
   15 | 00m11s |    0.84174 |      10.0000 |       14.0000 | 
   16 | 00m12s |    0.82773 |       7.4224 |       14.0000 | 
   17 | 00m11s |    0.81092 |       7.9663 |        7.5559 | 
   18 | 00m10s |    0.81933 |       1.0000 |       14.0000 | 


/anaconda2/envs/matias/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-4.32327801e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 56, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)


   19 | 00m15s |    0.83053 |       9.9999 |        8.8992 | 


/anaconda2/envs/matias/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-8.18825001e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 53, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)


   20 | 00m15s |    0.82353 |       1.0000 |        8.9973 | 
   21 | 00m14s |    0.82773 |       3.0472 |        7.5315 | 
   22 | 00m16s |    0.83193 |      10.0000 |       11.2614 | 


/anaconda2/envs/matias/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00018438]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 61, 'nit': 7, 'warnflag': 2}
  " state: %s" % convergence_dict)
/anaconda2/envs/matias/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00021916]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 52, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)
/anaconda2/envs/matias/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


   23 | 00m16s |    0.82773 |       3.3091 |       11.8331 | 
   24 | 00m16s |    0.82773 |       5.1381 |        8.3827 | 
   25 | 00m17s |    0.83053 |       5.3981 |       14.0000 | 


#### sacamos los mejores params

In [124]:
drill.best

{'max_val': 0.8417366943439516,
 'max_params': {'num_hidden': 10.0, 'num_neurons': 14.0}}